# Build subsurface model and forcing for Atlantis
This example shows how to build a subsurface model that has the correct input variables for subsidence modelling in [Atlantis Julia](https://gitlab.com/deltares/subsidence/atlans.jl). The model is built for an example area of 5x5 km nearby Utrecht. The area is shown in the figure below.

![Example area](./image/example_area.png)


# Input data
A complete model for the Netherlands and with the most level of detail is composed from the [BRO bodemkaart](), [GeoTOP](), [NL3D](), [AHN]() and a mean lowest groundwater table: the GLG. Below is an overview per data source:
- BRO bodemkaart: soilmap of the complete Netherlands documenting the subsurface buildup to a depth of 1.2 m below the surface level.
- GeoTOP: 3-dimensional voxelmodel documenting the stratigraphic and lithological subsurface buildup of the first 50 m below the surface level for most of the Netherlands (modelling for the rest of NL is still in progress). The model resolution is 100 x 100 x 0.5 [x, y, z].
- NL3D: 3-dimensional voxelmodel documenting the stratigraphic and lithological subsurface buildup to 20 m below the surface level for the entire Netherlands. The model resolution is 250 x 250 x 1 [x, y, z] and in contrast with GeoTOP, does not differentiate between Holocene units (i.e. Holocene is a single unit).
- AHN (Algemeen Hoogtebestand Nederland): Lidar surface level measurements for the entire Netherlands (excl. water bodies) with 0.5 x 0.5 m resolution.
- GLG: A mean lowest groundwater table. Input for this usually comes from national scale groundwater models. A national GLG is also openly available via the [BRO](https://basisregistratieondergrond.nl/inhoud-bro/registratieobjecten/modellen/model-grondwaterspiegeldiepte-wdm/).

To build a subsurface model suitable for Atlantis, AHN and GeoTOP are mandatory input. The other sources are optional. The area of this example is entirely covered by GeoTOP so only AHN, GeoTOP and the BRO Bodemkaart will be used. A GLG is thus optional and can be added at any time to the NetCDF of the eventual subsurface model.

It is not possible to access the BRO bodemkaart directly so this must first be downloaded as a geopackage from the link at the top of the section. Also a selection of AHN must be downloaded (see link at the top) and converted to a 100 x 100 m grid. GeoTOP data can directly be accessed from an OPeNDAP server.

### Build a model
Below, the necessary tools are imported and the data to build the model with are instantiated.

In [9]:
from atmod import (
    AtlansParameters,
    build_atlantis_model,
    read_ahn
)
from atmod.bro_models import BroBodemKaart, GeoTop, Lithology
from atmod.build_forcings import surcharge_like
from pathlib import Path


bbox  = (127_000, 448_000, 132_000, 453_000) # xmin, ymin, xmax, ymax of the example area

path_to_soilmap = 'bro_bodemkaart.gpkg'
path_to_ahn = 'dtm_100m.tif'

path_to_soilmap = r'c:\Users\knaake\OneDrive - Stichting Deltares\Documents\data\dino\bro_bodemkaart.gpkg'
path_to_ahn = r'p:\430-tgg-data\ahn\dtm_100m.tif'

ahn = read_ahn(path_to_ahn, bbox=bbox)
soilmap = BroBodemKaart.from_geopackage(path_to_soilmap, bbox=bbox)
geotop = GeoTop.from_opendap(bbox=bbox, data_vars=['strat', 'lithok'], lazy=False)

Load data


The above reads all data sources in the extent of the example area. GeoTOP is thus directly accessed from the OPeNDAP server, the method has the required url for the data. Only the 'strat' and 'lithok' variables of GeoTOP are downloaded as these are only required to build a subsurface model. Additionally, the 'lazy' keyword is set to False. This reads all data into memory which decreases the time it costs to build a complete model. Note that if the model area is larger, that a devices memory may not be sufficient and a model must be built in parts.

Building a model also needs a selection of input paramaters for parameterizing several data variables. Atmod has a class `AtlansParameters` where each input parameter can be specified. If a parameter is not specified, a default parameter is used. This example uses only defaults. The output shows the optional input parameters.

In [3]:
parameters = AtlansParameters() # initialize with all default parameters
print(parameters)

AtlansParameters:
	modelbase: -30.0
	mass_fraction_organic: 0.5
	mass_fraction_lutum: 0.5
	rho_bulk: 833.0
	shrinkage_degree: 0.7
	max_oxidation_depth: 1.2
	no_oxidation_thickness: 0.3
	no_shrinkage_thickness: 0.0


Finally, building a model is quite simple. The end result is an Xarray Dataset with the necessary input variables except a phreatic level.

In [4]:
model = build_atlantis_model(ahn, geotop, bodemkaart=soilmap, parameters=parameters)
print(model)

<xarray.Dataset> Size: 4MB
Dimensions:                 (x: 50, y: 50, layer: 76)
Coordinates:
  * x                       (x) float64 400B 1.27e+05 1.272e+05 ... 1.32e+05
  * y                       (y) float64 400B 4.53e+05 4.528e+05 ... 4.48e+05
  * layer                   (layer) int32 304B 1 2 3 4 5 6 ... 71 72 73 74 75 76
Data variables:
    geology                 (y, x, layer) float32 760kB 2.0 2.0 2.0 ... nan nan
    lithology               (y, x, layer) float32 760kB 8.0 8.0 8.0 ... nan nan
    thickness               (y, x, layer) float32 760kB 0.5 0.5 0.5 ... nan nan
    mass_fraction_organic   (y, x, layer) float32 760kB 0.0 0.0 0.0 ... nan nan
    surface                 (y, x) float32 10kB 2.869 -0.5485 ... 1.681 1.622
    rho_bulk                (y, x, layer) float32 760kB 833.0 833.0 ... nan nan
    zbase                   (y, x) float32 10kB -30.0 -30.0 ... -30.0 -30.0
    max_oxidation_depth     (y, x) float32 10kB 1.2 1.2 1.2 1.2 ... 1.2 1.2 1.2
    no_oxidation_thic

A 'phreatic_level' can be added to the NetCDF from any GLG input. The example here shows how to add a 'dummy' GLG at a constant depth 1.2 meters below the surface level. Note that the name 'phreatic_level' is mandatory for Atlantis Julia. All other names that are shown in the model are mandatory names too.

In [7]:
glg = model['surface'] - 1.2
model['phreatic_level'] = glg
print(model)

<xarray.Dataset> Size: 4MB
Dimensions:                 (x: 50, y: 50, layer: 76)
Coordinates:
  * x                       (x) float64 400B 1.27e+05 1.272e+05 ... 1.32e+05
  * y                       (y) float64 400B 4.53e+05 4.528e+05 ... 4.48e+05
  * layer                   (layer) int32 304B 1 2 3 4 5 6 ... 71 72 73 74 75 76
Data variables:
    geology                 (y, x, layer) float32 760kB 2.0 2.0 2.0 ... nan nan
    lithology               (y, x, layer) float32 760kB 8.0 8.0 8.0 ... nan nan
    thickness               (y, x, layer) float32 760kB 0.5 0.5 0.5 ... nan nan
    mass_fraction_organic   (y, x, layer) float32 760kB 0.0 0.0 0.0 ... nan nan
    surface                 (y, x) float32 10kB 2.869 -0.5485 ... 1.681 1.622
    rho_bulk                (y, x, layer) float32 760kB 833.0 833.0 ... nan nan
    zbase                   (y, x) float32 10kB -30.0 -30.0 ... -30.0 -30.0
    max_oxidation_depth     (y, x) float32 10kB 1.2 1.2 1.2 1.2 ... 1.2 1.2 1.2
    no_oxidation_thic

### Build a forcing
Several forcings can be applied during the modelling in Atlantis. Atmod provides easy functionality (not for every forcing yet) to build required NetCDF input data based on the extent of the subsurface model area. Forcings that can be applied in Atlantis are (see [Atlans.jl](https://gitlab.com/deltares/subsidence/atlans.jl) for an explanation of the forcings):
- Deep subsidence (not yet implemented in atmod)
- Stage indexation
- Stage change (not yet implemented in atmod)
- Aquifer head (not yet implemented in atmod)
- Temperature (not yet implemented in atmod)
- Surcharge

The example below shows how to easily create an Xarray Dataset for the surcharge forcing. Surcharge can be added in two ways shown below:
1. Add a cell with a specific lithology and thickness uniformly over the complete model area


In [10]:
import numpy as np

lithology = Lithology.fine_sand
thickness = 0.5

# Use four timesteps for example
timesteps = np.arange('2020-01-01', '2024-01-01', dtype='datetime64[Y]')

surcharge = surcharge_like(model, lithology, thickness, timesteps)
print(surcharge)

<xarray.Dataset> Size: 121kB
Dimensions:    (time: 4, layer: 1, y: 50, x: 50)
Coordinates:
  * time       (time) datetime64[ns] 32B 2020-01-01 2021-01-01 ... 2023-01-01
  * layer      (layer) int32 4B 1
  * y          (y) float64 400B 4.53e+05 4.528e+05 ... 4.482e+05 4.48e+05
  * x          (x) float64 400B 1.27e+05 1.272e+05 ... 1.318e+05 1.32e+05
Data variables:
    lithology  (time, layer, y, x) int32 40kB 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5
    thickness  (time, layer, y, x) float64 80kB 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5


2. Add a sequence of lithologies with corresponding thicknesses uniformly over the complete model area

In [17]:
lithology = np.array([Lithology.coarse_sand, Lithology.fine_sand])
thickness = np.array([0.5, 0.25])

surcharge = surcharge_like(model, lithology, thickness, timesteps) # Use the timesteps of example 1
print(surcharge)

<xarray.Dataset> Size: 241kB
Dimensions:    (time: 4, layer: 2, y: 50, x: 50)
Coordinates:
  * time       (time) datetime64[ns] 32B 2020-01-01 2021-01-01 ... 2023-01-01
  * layer      (layer) int32 8B 1 2
  * y          (y) float64 400B 4.53e+05 4.528e+05 ... 4.482e+05 4.48e+05
  * x          (x) float64 400B 1.27e+05 1.272e+05 ... 1.318e+05 1.32e+05
Data variables:
    lithology  (time, layer, y, x) int32 80kB 7 7 7 7 7 7 7 7 ... 5 5 5 5 5 5 5
    thickness  (time, layer, y, x) float64 160kB 0.5 0.5 0.5 ... 0.25 0.25 0.25
